In [ ]:
%load_ext autoreload
%autoreload

from os import getcwd
from os.path import join, abspath, pardir, relpath, exists

import requests

from dataclasses import dataclass, field

import dask.dataframe as dd
import pandas as pd
import numpy as np
from numpy import matrixlib as npmat
import networkx as nx
from typing import Union
import pulp as p
from itertools import combinations
from typing import List, Tuple
from enum import Enum
from scipy.stats import kendalltau, spearmanr

from IPython.display import IFrame
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

: 

In [ ]:
pd.set_option('display.max_columns', None)
parent_dir = abspath(join(getcwd(), pardir))
data_dir = join(parent_dir, 'data')
data_file = join(data_dir, 'products1.csv')
parent_dir

: 

In [8]:
df = pd.read_csv(data_file)
df.head()

,id,name,image_url,brand,categories,energy,energy_points,energy_value,fiber,fiber_points,fiber_value,fruits_vegetables_nuts_colza_walnut_olive_oils,fruits_vegetables_nuts_colza_walnut_olive_oils_points,fruits_vegetables_nuts_colza_walnut_olive_oils_value,grade,is_beverage,is_cheese,is_fat,is_water,negative_points,positive_points,proteins,proteins_points,proteins_value,saturated_fat,saturated_fat_points,saturated_fat_ratio,saturated_fat_ratio_points,saturated_fat_ratio_value,saturated_fat_value,score,sodium,sodium_points,sodium_value,sugars,sugars_points,sugars_value,carbohydrates,carbohydrates_100g,carbohydrates_unit,carbohydrates_value,energy-kcal,energy-kcal_100g,energy-kcal_unit,energy-kcal_value,energy-kcal_value_computed,energy_100g,energy_unit,fat,fat_100g,fat_unit,fat_value,fruits-vegetables-nuts-estimate-from-ingredients_100g,fruits-vegetables-nuts-estimate-from-ingredients_serving,nova-group,nova-group_100g,nova-group_serving,nutrition-score-fr,nutrition-score-fr_100g,proteins_100g,proteins_unit,salt,salt_100g,salt_unit,salt_value,saturated-fat,saturated-fat_100g,saturated-fat_unit,saturated-fat_value,sodium_100g,sodium_unit,sugars_100g,sugars_unit,carbohydrates_serving,energy-kcal_serving,energy-kj,energy-kj_100g,energy-kj_serving,energy-kj_unit,energy-kj_value,energy-kj_value_computed,energy_serving,fat_serving,fiber_100g,fiber_serving,fiber_unit,proteins_serving,salt_serving,saturated-fat_serving,sodium_serving,starch,starch_100g,starch_label,starch_serving,starch_unit,starch_value,sugars_serving,carbon-footprint-from-known-ingredients_100g,carbon-footprint-from-known-ingredients_product,carbon-footprint-from-known-ingredients_serving,fruits-vegetables-nuts-estimate,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate_label,fruits-vegetables-nuts-estimate_serving,fruits-vegetables-nuts-estimate_unit,fruits-vegetables-nuts-estimate_value,alcohol,alcohol_100g,alcohol_serving,alcohol_unit,alcohol_value,vitamin-b1,vitamin-b12,vitamin-b12_100g,vitamin-b12_label,vitamin-b12_serving,vitamin-b12_unit,vitamin-b12_value,vitamin-b1_100g,vitamin-b1_label,vitamin-b1_serving,vitamin-b1_unit,vitamin-b1_value,vitamin-b2,vitamin-b2_100g,vitamin-b2_label,vitamin-b2_serving,vitamin-b2_unit,vitamin-b2_value,vitamin-b9,vitamin-b9_100g,vitamin-b9_label,vitamin-b9_serving,vitamin-b9_unit,vitamin-b9_value,vitamin-pp,vitamin-pp_100g,vitamin-pp_label,vitamin-pp_serving,vitamin-pp_unit,vitamin-pp_value,fruits-vegetables-nuts,fruits-vegetables-nuts_100g,fruits-vegetables-nuts_serving,fruits-vegetables-nuts_unit,fruits-vegetables-nuts_value,carbon-footprint-from-meat-or-fish_100g,carbon-footprint-from-meat-or-fish_product,carbon-footprint-from-meat-or-fish_serving,iron,iron_100g,iron_label,iron_serving,iron_unit,iron_value,magnesium,magnesium_100g,magnesium_label,magnesium_serving,magnesium_unit,magnesium_value,phosphorus,phosphorus_100g,phosphorus_label,phosphorus_serving,phosphorus_unit,phosphorus_value,zinc,zinc_100g,zinc_label,zinc_serving,zinc_unit,zinc_value,fiber_modifier,calcium,calcium_100g,calcium_serving,calcium_unit,calcium_value,vitamin-d,vitamin-d_100g,vitamin-d_label,vitamin-d_serving,vitamin-d_unit,vitamin-d_value,vitamin-e,vitamin-e_100g,vitamin-e_label,vitamin-e_serving,vitamin-e_unit,vitamin-e_value,fruits-vegetables-nuts-dried,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-dried_serving,fruits-vegetables-nuts-dried_unit,fruits-vegetables-nuts-dried_value,glucides assimilables,glucides assimilables_100g,glucides assimilables_serving,glucides assimilables_unit,glucides assimilables_value,vitamin-c,vitamin-c_100g,vitamin-c_label,vitamin-c_serving,vitamin-c_unit,vitamin-c_value,saturated-fat_modifier,nutriscore_not_applicable_for_category,monounsaturated-fat,monounsaturated-fat_100g,monounsaturated-fat_serving,monounsaturated-fat_unit,monounsaturated-fat_value,polyunsaturated-fat,polyunsaturated-fat_100g,polyunsaturated-fat_serving,polyunsaturated-fat_unit,polyunsaturated-fat_value,salt_modifier,sodium

In [9]:
df.columns.to_list()

['id',
 'name',
 'image_url',
 'brand',
 'categories',
 'energy',
 'energy_points',
 'energy_value',
 'fiber',
 'fiber_points',
 'fiber_value',
 'fruits_vegetables_nuts_colza_walnut_olive_oils',
 'fruits_vegetables_nuts_colza_walnut_olive_oils_points',
 'fruits_vegetables_nuts_colza_walnut_olive_oils_value',
 'grade',
 'is_beverage',
 'is_cheese',
 'is_fat',
 'is_water',
 'negative_points',
 'positive_points',
 'proteins',
 'proteins_points',
 'proteins_value',
 'saturated_fat',
 'saturated_fat_points',
 'saturated_fat_ratio',
 'saturated_fat_ratio_points',
 'saturated_fat_ratio_value',
 'saturated_fat_value',
 'score',
 'sodium',
 'sodium_points',
 'sodium_value',
 'sugars',
 'sugars_points',
 'sugars_value',
 'carbohydrates',
 'carbohydrates_100g',
 'carbohydrates_unit',
 'carbohydrates_value',
 'energy-kcal',
 'energy-kcal_100g',
 'energy-kcal_unit',
 'energy-kcal_value',
 'energy-kcal_value_computed',
 'energy_100g',
 'energy_unit',
 'fat',
 'fat_100g',
 'fat_unit',
 'fat_value',
 

In [10]:
df[df["negative_points"] != (df["energy_points"] + df["saturated_fat_points"] + df["sugars_points"] + df["sodium_points"])]

,id,name,image_url,brand,categories,energy,energy_points,energy_value,fiber,fiber_points,fiber_value,fruits_vegetables_nuts_colza_walnut_olive_oils,fruits_vegetables_nuts_colza_walnut_olive_oils_points,fruits_vegetables_nuts_colza_walnut_olive_oils_value,grade,is_beverage,is_cheese,is_fat,is_water,negative_points,positive_points,proteins,proteins_points,proteins_value,saturated_fat,saturated_fat_points,saturated_fat_ratio,saturated_fat_ratio_points,saturated_fat_ratio_value,saturated_fat_value,score,sodium,sodium_points,sodium_value,sugars,sugars_points,sugars_value,carbohydrates,carbohydrates_100g,carbohydrates_unit,carbohydrates_value,energy-kcal,energy-kcal_100g,energy-kcal_unit,energy-kcal_value,energy-kcal_value_computed,energy_100g,energy_unit,fat,fat_100g,fat_unit,fat_value,fruits-vegetables-nuts-estimate-from-ingredients_100g,fruits-vegetables-nuts-estimate-from-ingredients_serving,nova-group,nova-group_100g,nova-group_serving,nutrition-score-fr,nutrition-score-fr_100g,proteins_100g,proteins_unit,salt,salt_100g,salt_unit,salt_value,saturated-fat,saturated-fat_100g,saturated-fat_unit,saturated-fat_value,sodium_100g,sodium_unit,sugars_100g,sugars_unit,carbohydrates_serving,energy-kcal_serving,energy-kj,energy-kj_100g,energy-kj_serving,energy-kj_unit,energy-kj_value,energy-kj_value_computed,energy_serving,fat_serving,fiber_100g,fiber_serving,fiber_unit,proteins_serving,salt_serving,saturated-fat_serving,sodium_serving,starch,starch_100g,starch_label,starch_serving,starch_unit,starch_value,sugars_serving,carbon-footprint-from-known-ingredients_100g,carbon-footprint-from-known-ingredients_product,carbon-footprint-from-known-ingredients_serving,fruits-vegetables-nuts-estimate,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate_label,fruits-vegetables-nuts-estimate_serving,fruits-vegetables-nuts-estimate_unit,fruits-vegetables-nuts-estimate_value,alcohol,alcohol_100g,alcohol_serving,alcohol_unit,alcohol_value,vitamin-b1,vitamin-b12,vitamin-b12_100g,vitamin-b12_label,vitamin-b12_serving,vitamin-b12_unit,vitamin-b12_value,vitamin-b1_100g,vitamin-b1_label,vitamin-b1_serving,vitamin-b1_unit,vitamin-b1_value,vitamin-b2,vitamin-b2_100g,vitamin-b2_label,vitamin-b2_serving,vitamin-b2_unit,vitamin-b2_value,vitamin-b9,vitamin-b9_100g,vitamin-b9_label,vitamin-b9_serving,vitamin-b9_unit,vitamin-b9_value,vitamin-pp,vitamin-pp_100g,vitamin-pp_label,vitamin-pp_serving,vitamin-pp_unit,vitamin-pp_value,fruits-vegetables-nuts,fruits-vegetables-nuts_100g,fruits-vegetables-nuts_serving,fruits-vegetables-nuts_unit,fruits-vegetables-nuts_value,carbon-footprint-from-meat-or-fish_100g,carbon-footprint-from-meat-or-fish_product,carbon-footprint-from-meat-or-fish_serving,iron,iron_100g,iron_label,iron_serving,iron_unit,iron_value,magnesium,magnesium_100g,magnesium_label,magnesium_serving,magnesium_unit,magnesium_value,phosphorus,phosphorus_100g,phosphorus_label,phosphorus_serving,phosphorus_unit,phosphorus_value,zinc,zinc_100g,zinc_label,zinc_serving,zinc_unit,zinc_value,fiber_modifier,calcium,calcium_100g,calcium_serving,calcium_unit,calcium_value,vitamin-d,vitamin-d_100g,vitamin-d_label,vitamin-d_serving,vitamin-d_unit,vitamin-d_value,vitamin-e,vitamin-e_100g,vitamin-e_label,vitamin-e_serving,vitamin-e_unit,vitamin-e_value,fruits-vegetables-nuts-dried,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-dried_serving,fruits-vegetables-nuts-dried_unit,fruits-vegetables-nuts-dried_value,glucides assimilables,glucides assimilables_100g,glucides assimilables_serving,glucides assimilables_unit,glucides assimilables_value,vitamin-c,vitamin-c_100g,vitamin-c_label,vitamin-c_serving,vitamin-c_unit,vitamin-c_value,saturated-fat_modifier,nutriscore_not_applicable_for_category,monounsaturated-fat,monounsaturated-fat_100g,monounsaturated-fat_serving,monounsaturated-fat_unit,monounsaturated-fat_value,polyunsaturated-fat,polyunsaturated-fat_100g,polyunsaturated-fat_serving,polyunsaturated-fat_unit,polyunsaturated-fat_value,salt_modifier,sodium

In [11]:
df[df["id"] == 8480024002679]

,id,name,image_url,brand,categories,energy,energy_points,energy_value,fiber,fiber_points,fiber_value,fruits_vegetables_nuts_colza_walnut_olive_oils,fruits_vegetables_nuts_colza_walnut_olive_oils_points,fruits_vegetables_nuts_colza_walnut_olive_oils_value,grade,is_beverage,is_cheese,is_fat,is_water,negative_points,positive_points,proteins,proteins_points,proteins_value,saturated_fat,saturated_fat_points,saturated_fat_ratio,saturated_fat_ratio_points,saturated_fat_ratio_value,saturated_fat_value,score,sodium,sodium_points,sodium_value,sugars,sugars_points,sugars_value,carbohydrates,carbohydrates_100g,carbohydrates_unit,carbohydrates_value,energy-kcal,energy-kcal_100g,energy-kcal_unit,energy-kcal_value,energy-kcal_value_computed,energy_100g,energy_unit,fat,fat_100g,fat_unit,fat_value,fruits-vegetables-nuts-estimate-from-ingredients_100g,fruits-vegetables-nuts-estimate-from-ingredients_serving,nova-group,nova-group_100g,nova-group_serving,nutrition-score-fr,nutrition-score-fr_100g,proteins_100g,proteins_unit,salt,salt_100g,salt_unit,salt_value,saturated-fat,saturated-fat_100g,saturated-fat_unit,saturated-fat_value,sodium_100g,sodium_unit,sugars_100g,sugars_unit,carbohydrates_serving,energy-kcal_serving,energy-kj,energy-kj_100g,energy-kj_serving,energy-kj_unit,energy-kj_value,energy-kj_value_computed,energy_serving,fat_serving,fiber_100g,fiber_serving,fiber_unit,proteins_serving,salt_serving,saturated-fat_serving,sodium_serving,starch,starch_100g,starch_label,starch_serving,starch_unit,starch_value,sugars_serving,carbon-footprint-from-known-ingredients_100g,carbon-footprint-from-known-ingredients_product,carbon-footprint-from-known-ingredients_serving,fruits-vegetables-nuts-estimate,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate_label,fruits-vegetables-nuts-estimate_serving,fruits-vegetables-nuts-estimate_unit,fruits-vegetables-nuts-estimate_value,alcohol,alcohol_100g,alcohol_serving,alcohol_unit,alcohol_value,vitamin-b1,vitamin-b12,vitamin-b12_100g,vitamin-b12_label,vitamin-b12_serving,vitamin-b12_unit,vitamin-b12_value,vitamin-b1_100g,vitamin-b1_label,vitamin-b1_serving,vitamin-b1_unit,vitamin-b1_value,vitamin-b2,vitamin-b2_100g,vitamin-b2_label,vitamin-b2_serving,vitamin-b2_unit,vitamin-b2_value,vitamin-b9,vitamin-b9_100g,vitamin-b9_label,vitamin-b9_serving,vitamin-b9_unit,vitamin-b9_value,vitamin-pp,vitamin-pp_100g,vitamin-pp_label,vitamin-pp_serving,vitamin-pp_unit,vitamin-pp_value,fruits-vegetables-nuts,fruits-vegetables-nuts_100g,fruits-vegetables-nuts_serving,fruits-vegetables-nuts_unit,fruits-vegetables-nuts_value,carbon-footprint-from-meat-or-fish_100g,carbon-footprint-from-meat-or-fish_product,carbon-footprint-from-meat-or-fish_serving,iron,iron_100g,iron_label,iron_serving,iron_unit,iron_value,magnesium,magnesium_100g,magnesium_label,magnesium_serving,magnesium_unit,magnesium_value,phosphorus,phosphorus_100g,phosphorus_label,phosphorus_serving,phosphorus_unit,phosphorus_value,zinc,zinc_100g,zinc_label,zinc_serving,zinc_unit,zinc_value,fiber_modifier,calcium,calcium_100g,calcium_serving,calcium_unit,calcium_value,vitamin-d,vitamin-d_100g,vitamin-d_label,vitamin-d_serving,vitamin-d_unit,vitamin-d_value,vitamin-e,vitamin-e_100g,vitamin-e_label,vitamin-e_serving,vitamin-e_unit,vitamin-e_value,fruits-vegetables-nuts-dried,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-dried_serving,fruits-vegetables-nuts-dried_unit,fruits-vegetables-nuts-dried_value,glucides assimilables,glucides assimilables_100g,glucides assimilables_serving,glucides assimilables_unit,glucides assimilables_value,vitamin-c,vitamin-c_100g,vitamin-c_label,vitamin-c_serving,vitamin-c_unit,vitamin-c_value,saturated-fat_modifier,nutriscore_not_applicable_for_category,monounsaturated-fat,monounsaturated-fat_100g,monounsaturated-fat_serving,monounsaturated-fat_unit,monounsaturated-fat_value,polyunsaturated-fat,polyunsaturated-fat_100g,polyunsaturated-fat_serving,polyunsaturated-fat_unit,polyunsaturated-fat_value,salt_modifier,sodium